# LIBRARY

In [28]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.linear_model import LinearRegression
from statsmodels.formula.api import ols
from statsmodels.formula.api import glm
from statsmodels.genmod.families.family import Binomial
from sklearn.ensemble import RandomForestClassifier

In [29]:
import warnings
warnings.filterwarnings('ignore')

# DATA

In [30]:
train = pd.read_csv('train_PCA_fill_ar.csv')
val = pd.read_csv('valid_PCA_fill_ar.csv')
test = pd.read_csv('test_PCA_fill_ar.csv')
sub = pd.read_csv('../Data/PJT002_submission.csv')

### 필요한 열만 가져오기

In [31]:
# columns = ['id', 'dt_of_fr', 'bldng_cnt', 'bldng_ar', 'ttl_ar', 'lnd_ar'
#            , 'jmk', 'fr_sttn_dstnc', 'fr_wthr_fclt_dstnc'
#            , 'mlt_us_yn', 'cctv_dstnc', 'cctv_in_100m'
#            , 'fr_wthr_fclt_in_100m', 'tbc_rtl_str_dstnc', 'sft_emrgnc_bll_dstnc'
#            , 'ahsm_dstnc', 'no_tbc_zn_dstnc', 'bldng_cnt_in_50m', 'fr_yn'
#           ]
# columns2 = ['id', 'dt_of_fr', 'bldng_cnt', 'bldng_ar', 'ttl_ar', 'lnd_ar'
#            , 'jmk', 'fr_sttn_dstnc', 'fr_wthr_fclt_dstnc'
#            , 'mlt_us_yn', 'cctv_dstnc', 'cctv_in_100m'
#            , 'fr_wthr_fclt_in_100m', 'tbc_rtl_str_dstnc', 'sft_emrgnc_bll_dstnc'
#            , 'ahsm_dstnc', 'no_tbc_zn_dstnc', 'bldng_cnt_in_50m'
#           ]

# train_pp = train[columns]
# test_pp = test[columns2]
# val_pp = val[columns]

# NULL CHECK
null 없는 애들로만 가져와서 null이 없음

In [32]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59191 entries, 0 to 59190
Data columns (total 93 columns):
address                  59191 non-null object
address1                 59191 non-null object
address2                 59191 non-null object
ahsm_dstnc               59191 non-null int64
bldng_ar                 0 non-null float64
bldng_cnt                59191 non-null int64
bldng_cnt_in_50m         59191 non-null int64
cctv_dstnc               59191 non-null int64
cctv_in_100m             59191 non-null int64
day                      59191 non-null int64
dayofweek                59191 non-null object
fire_yes                 59191 non-null float64
fr_mn_cnt                59191 non-null float64
fr_sttn_dstnc            59191 non-null int64
fr_wthr_fclt_dstnc       59191 non-null int64
fr_wthr_fclt_in_100m     59191 non-null int64
hm_cnt                   59191 non-null float64
hmdt                     59191 non-null float64
hour                     59191 non-null int64
id     

In [33]:
train = train[train['address1'] != '창원시']

In [34]:
train['address1'] != '창원시'

0        True
1        True
2        True
3        True
4        True
         ... 
59186    True
59187    True
59188    True
59189    True
59190    True
Name: address1, Length: 50749, dtype: bool

# MAKING DATASET

In [36]:
# choose independent variables(Xs) which are useful!
# 트레이닝에 사용할 변수 목록을 적어주세요.
independents = [
     'bldng_cnt', 'bldng_ar', 'ttl_ar', 'lnd_ar', 'ttl_dwn_flr', 'ttl_grnd_flr',
#       'jmk',
#    'address1', 
    'fr_wthr_fclt_in_100m', 'mlt_us_yn(encode)',
       'tbc_rtl_str_dstnc',
    
#     'fr_sttn_dstnc', 'fr_wthr_fclt_dstnc',  'cctv_dstnc',
#        'cctv_in_100m', 
#        'sft_emrgnc_bll_dstnc', 'ahsm_dstnc', 'no_tbc_zn_dstnc',
#        'bldng_cnt_in_50m',  
    'month',
#         'year-month'
] 
# 독립변수
dependent = ['fire_yes'] # 종속변수

In [37]:
# 적어준 변수 목록을 사용해 데이터를 트레이닝에 맞는 포맷으로 변경합니다.
train_X = train[independents]
train_y = train[dependent]

test_X = test[independents]
val_X = val[independents]

# MODEL FITTING

## 선형 회귀

In [39]:
reg = LinearRegression().fit(train_X, train_y)
# 대문자는 매트릭스, 소문자는 벡터

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [40]:
# x 계수
# x 계수의 의미를 해석하세요.
reg.coef_
# 피클래스가 한 계단 올라갈 때마다 죽음에 0.19배 가까워짐

NameError: name 'reg' is not defined

In [41]:
# 절편
reg.intercept_

NameError: name 'reg' is not defined

In [42]:
reg.score(train_X, train_y, sample_weight=None)

NameError: name 'reg' is not defined

## 일반화 선형모형

In [43]:
res = glm('fire_yes ~ fr_wthr_fclt_in_100m + tbc_rtl_str_dstnc + month', train, family=Binomial()).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:               fire_yes   No. Observations:                50749
Model:                            GLM   Df Residuals:                    50745
Model Family:                Binomial   Df Model:                            3
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -21121.
Date:                Wed, 13 Nov 2019   Deviance:                       42242.
Time:                        12:30:25   Pearson chi2:                 5.12e+04
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -1.3375      0.030    -43.861      0.000      -1.397      -1.278
fr_wthr_fclt_in_100m     0.1399      0.018      7.935      0.000       0.105       0.174
tbc_rtl_str_dstnc    -9.892e-05   4.52e-06    -21.866      0.000      -0.000   -9.01e-05
month                   -0.0190      0.004     -5.299      0.000      -0.026      -0.012
========================================================================================
"""

In [44]:
res = glm('fire_yes ~ C(address1)*C(jmk)', train, family=Binomial()).fit()
res.summary()

KeyboardInterrupt: 

## 최소제곱추정 선형회귀

In [45]:
res = ols('fire_yes ~ month', train).fit()
# res.summary()

In [46]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               fire_yes   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     25.33
Date:                Wed, 13 Nov 2019   Prob (F-statistic):           4.86e-07
Time:                        12:33:18   Log-Likelihood:                -19846.
No. Observations:               50749   AIC:                         3.970e+04
Df Residuals:                   50747   BIC:                         3.971e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1656      0.003     49.496      0.000       0.159       0.172
month         -0.0023      0.000     -5.032      0.000      -0.003      -0.001
==============================================================================
Omnibus:                    16624.062   Durbin-Watson:                   1.926
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            39076.087
Skew:                           1.950   Prob(JB):                         0.00
Kurtosis:                       4.807   Cond. No.                         15.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## 값 예측하기

In [47]:
prediction = res.predict(test_X)

In [48]:
sorted(prediction, reverse=True)

[0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.16333266233993854,
 0.1633326

In [49]:
prediction_binomial = [1 if p>=0.3 else 0 for p in prediction]

# VALIDATION 파일 이용하기

In [41]:
prediction_val = res.predict(val_X)
sorted(prediction_val, reverse=True)

[0.9068171198939408,
 0.9064486786279641,
 0.9051466644158523,
 0.8996389810845806,
 0.896407473646308,
 0.8952653999191234,
 0.8951169006193903,
 0.8949053019823713,
 0.8947468614743204,
 0.8947097671189543,
 0.8934003818588635,
 0.8933203662912034,
 0.8921100954134039,
 0.8914967894360755,
 0.8910174269828216,
 0.8900312957794704,
 0.8897937712551934,
 0.8891836473146604,
 0.8890786825164142,
 0.889010376176495,
 0.8887785778932198,
 0.8881560319657672,
 0.8881093944443159,
 0.8878558843028926,
 0.8874638196242107,
 0.887003698104928,
 0.887003698104928,
 0.8869687123121752,
 0.8864049102185289,
 0.8855177684424669,
 0.8849148833068851,
 0.8848542347264456,
 0.8847005854454982,
 0.8844532317759597,
 0.8843671794060972,
 0.8843445593217321,
 0.8839772421638609,
 0.8838072440143023,
 0.8837181847542089,
 0.883634056513132,
 0.8836087207269014,
 0.8835478570339992,
 0.8834827572190563,
 0.8832519826404156,
 0.8828740314161991,
 0.8828539560338912,
 0.8824183525777645,
 0.882418352577764

In [103]:
prediction_binomial_val = [1 if p>=0.9 else 0 for p in prediction_val]

# RANDOM FOREST

In [43]:
forest = RandomForestClassifier(random_state=42, n_estimators=10)

In [44]:
forest = forest.fit(train_X, train_y)

In [45]:
forest.feature_importances_

array([0.01556735, 0.00502622, 0.94084573, 0.03856071])

In [46]:
predict_forest = forest.predict(val_X)

In [47]:
predict_forest

array([False, False, False, ..., False, False, False])

In [65]:
# MSE
def mse(prediction, validation):
    return np.mean((prediction - validation) ** 2) 

In [104]:
mse(prediction_binomial_val, val['fire_yes'])

0.18454624528848942

In [69]:
mse([0] * len(val['fire_yes']), val['fire_yes'])

0.18440127573209625

# F1 구하기 Precision/Recall

In [48]:
import sklearn.metrics as metrics

In [52]:
print('f1', metrics.f1_score(prediction_binomial_val,val['fire_yes']) )

f1 0.3113831089351285


In [54]:
print('f1', metrics.f1_score(predict_forest,val['fire_yes']) )

f1 0.17750439367311072


# 한 번에 모델 돌리기

In [55]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

In [56]:
def train_and_test(model):
    a = model.fit(train_X, train_y)
    prediction = a.predict(test_X)
    accuracy = round(model.score(train_X, train_y) * 100, 2)
    print("Accuracy : ", accuracy, "%")
    return prediction

In [33]:
log_pred = train_and_test(LogisticRegression())

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy :  87.5 %


In [34]:
# svm_pred = train_and_test(SVC())

In [35]:
# knn_pred_4 = train_and_test(KNeighborsClassifier(n_neighbors = 4))

In [36]:
rf_pred = train_and_test(RandomForestClassifier(n_estimators=100))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


Accuracy :  99.94 %


In [37]:
# nb_pred = train_and_test(GaussianNB())

# VALIDATION 한 번에 돌리기

In [85]:
def train_and_val(model):
    a = model.fit(train_X, train_y)
    prediction_val = a.predict(val_X)
    prediction_binomial_val = [1 if p>=0.3 else 0 for p in prediction_val]
    print('f1:', metrics.f1_score(prediction_binomial_val,val_pp['fr_yn']) )
    return prediction_val

In [86]:
train_and_val(LogisticRegression())

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


f1: 0.20277777777777778


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [87]:
train_and_val(RandomForestClassifier(n_estimators=100))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


f1: 0.383634431455898


array([1, 0, 0, ..., 0, 0, 0], dtype=int64)

In [88]:
train_and_val(KNeighborsClassifier(n_neighbors = 4))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


f1: 0.31120552310143385


array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

In [89]:
train_and_val(GaussianNB())

f1: 0.20207612456747404


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [ ]:
train_and_val(SVC())

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
